In [1]:
export cwd=$(pwd)

In [2]:
echo $cwd

/cluster/gs_lab/common/co_chip


In [3]:
export SRR_RUN_TABLE=$cwd/SraRunTable.txt

In [4]:
cd $cwd
mkdir run
cd run

mkdir: cannot create directory `run': File exists


In [5]:
rm -rf *

In [6]:
mkdir index
mkdir annotation

In [7]:
cd annotation
wget ftp://ftp.ensembl.org/pub/release-97/fasta/mus_musculus/dna//Mus_musculus.GRCm38.dna.toplevel.fa.gz
cd ..

--2019-08-20 15:32:11--  ftp://ftp.ensembl.org/pub/release-97/fasta/mus_musculus/dna//Mus_musculus.GRCm38.dna.toplevel.fa.gz
           => 'Mus_musculus.GRCm38.dna.toplevel.fa.gz'
Resolving ftp.ensembl.org... 193.62.193.8
Connecting to ftp.ensembl.org|193.62.193.8|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-97/fasta/mus_musculus/dna/ ... done.
==> SIZE Mus_musculus.GRCm38.dna.toplevel.fa.gz ... 842816838
==> PASV ... done.    ==> RETR Mus_musculus.GRCm38.dna.toplevel.fa.gz ... done.
Length: 842816838 (804M) (unauthoritative)

gz                    6%[>                   ]  53.24M  2.62MB/s    eta 4m 54s 


In [ ]:
cd index
bowtie2-build ../annotation/Mus_musculus.GRCm38.dna.toplevel.fa.gz mm9
bowtie2-inspect -n mm9
export INDEX=$(pwd)/mm9
cd ..

bash: cd: index: No such file or directory
Settings:
  Output files: "mm9.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ../annotation/Mus_musculus.GRCm38.dna.toplevel.fa.gz
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:03
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:04
bmax according to bmaxDivN setting: 46114548
Using parameters --bmax 34585911 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing

In [ ]:
python3 $cwd/parse_sra_runtable.py $SRR_RUN_TABLE

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 17 "$cwd/align.sh {} 2"

In [ ]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 40 $'samtools view -h {}/all.bam |awk "sqrt((\$9)^2) < 230" |\samtools view -bS - > {}/all-filtered.bam'

In [ ]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 40 "samtools index {}/all-filtered.bam"

In [ ]:
find ./*_*/SRX*/merged -mindepth 0 -maxdepth 0 | parallel -j 17 "bamPEFragmentSize -b {}/all-filtered.bam -o {}/hist.png --numberOfProcessors 2"

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 40 "makeTagDirectory {}/tag_dir {}/merged/all-filtered.bam"

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 40 "findPeaks {}/tag_dir -style histone -size 200 -minDist 100 -o auto"

In [ ]:
find ./*_*/SRX* -mindepth 0 -maxdepth 0 | parallel -j 40 $'awk "/^[^#]/ {print \$2,\$3,\$4,\$1,\$8,\$5}" {}/tag_dir/regions.txt > {}/tag_dir/regions.bed'